# 2-4-Stanの基本

In [ ]:
# -*- coding: utf-8 -*-
# R/StanコードのPython + NumPyro版
# - pandasでCSV読込
# - NumPyroでベイズ推定 (NUTS)
# - モデルの可視化: numpyro.render_model
# - トレース/事後: ArviZ (plot_posteriorはhdi_probを使用)
# - データの可視化: matplotlib
# - 計算結果の表示は print()

import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import arviz as az

import jax
import jax.numpy as jnp
from jax import random

import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
from numpyro import render_model

# -------------------------------
# 0) 設定（Rのrstan_optionsやmc.cores相当）
# -------------------------------
# 乱数シード (Rのseed=1に対応)
SEED = 1
rng_key = random.PRNGKey(SEED)

# チェーン数・反復数（Rコード準拠）
NUM_CHAINS = 4
NUM_WARMUP = 1000  # warmup/burn-in
NUM_SAMPLES = 1000 # iter=2000 - warmup=1000 と同等
THINNING = 1

# （任意）ホストのデバイス数をチェーン数に合わせると並列化しやすい
try:
    numpyro.set_host_device_count(NUM_CHAINS)
except Exception:
    pass

# -------------------------------
# 1) データ読み込み（pandas）
# -------------------------------
csv_path = "2-4-1-beer-sales-1.csv"
if not os.path.exists(csv_path):
    raise FileNotFoundError(
        f"CSV file not found: {csv_path}\n"
        "Place '2-4-1-beer-sales-1.csv' in the working directory."
    )

df = pd.read_csv(csv_path)

# データの確認（Rの head(..., n=3) 相当）
print("Head (first 3 rows):")
print(df.head(3))

# サンプルサイズ
sample_size = len(df)
print("\nSample size (N):")
print(sample_size)

# sales ベクトル（JAX配列）
if "sales" not in df.columns:
    raise KeyError("The CSV must contain a 'sales' column.")
sales = jnp.array(df["sales"].values)

# -------------------------------
# 2) 参考: データの可視化（matplotlib, 英語ラベル）
# -------------------------------
plt.figure()
plt.hist(np.asarray(sales), bins=20, edgecolor="white")
plt.title("Beer Sales Distribution")
plt.xlabel("Sales")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# -------------------------------
# 3) NumPyroモデル定義（Stanの2種）
#    - Stan(逐次版): forループで観測を一つずつ
#    - Stan(ベクトル化版): plateでベクトル化
#    ※ Stanコードは事前分布が明示されていない（不適切事前）ため、
#      NumPyroでは弱情報事前を与えます（近似的にフラット）。
# -------------------------------

# 逐次版（obs_0, obs_1, ... と観測ノードが並ぶ）
def model_loop(sales):
    N = sales.shape[0]
    # 弱情報事前（Stanのフラット近似）
    mu = numpyro.sample("mu", dist.Normal(0.0, 1000.0))
    sigma = numpyro.sample("sigma", dist.HalfCauchy(1000.0))
    for i in range(N):
        numpyro.sample(f"obs_{i}", dist.Normal(mu, sigma), obs=sales[i])

# ベクトル化版（plateで一括）
def model_vectorized(sales):
    N = sales.shape[0]
    mu = numpyro.sample("mu", dist.Normal(0.0, 1000.0))
    sigma = numpyro.sample("sigma", dist.HalfCauchy(1000.0))
    with numpyro.plate("data", N):
        numpyro.sample("obs", dist.Normal(mu, sigma), obs=sales)

# -------------------------------
# 4) モデルの可視化（NumPyro組み込み）
# -------------------------------
# Graphvizが無い環境でも落ちないようにtry
try:
    g1 = render_model(model_loop, model_args=(sales,), render_distributions=True)
    # g1.view()  # ファイル出力したい場合
    print("\nRendered model graph (loop) created.")
    g2 = render_model(model_vectorized, model_args=(sales,), render_distributions=True)
    # g2.view()
    print("Rendered model graph (vectorized) created.")
except Exception as e:
    print("\nModel rendering skipped (Graphviz may be missing):")
    print(repr(e))

# -------------------------------
# 5) MCMC（逐次版）
# -------------------------------
nuts_loop = NUTS(model_loop)
mcmc_loop = MCMC(
    nuts_loop,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    thinning=THINNING,
)
mcmc_loop.run(rng_key, sales=sales)
print("\n[Loop model] MCMC summary:")
mcmc_loop.print_summary(exclude_deterministic=False)

idata_loop = az.from_numpyro(mcmc_loop)

# 逐次版の要約（95% HDI, print利用）
print("\n[Loop model] ArviZ summary (95% HDI):")
print(az.summary(idata_loop, var_names=["mu", "sigma"], hdi_prob=0.95))

# トレースと事後分布（英語ラベル）
az.plot_trace(idata_loop, var_names=["mu", "sigma"])
plt.suptitle("Trace Plot (Loop Model)", y=1.02)
plt.tight_layout()
plt.show()

az.plot_posterior(idata_loop, var_names=["mu", "sigma"], hdi_prob=0.95)
plt.suptitle("Posterior (Loop Model, 95% HDI)", y=1.02)
plt.tight_layout()
plt.show()

# -------------------------------
# 6) MCMC（ベクトル化版）
# -------------------------------
rng_key2 = random.split(rng_key, 2)[1]

nuts_vec = NUTS(model_vectorized)
mcmc_vec = MCMC(
    nuts_vec,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    thinning=THINNING,
)
mcmc_vec.run(rng_key2, sales=sales)
print("\n[Vectorized model] MCMC summary:")
mcmc_vec.print_summary(exclude_deterministic=False)

idata_vec = az.from_numpyro(mcmc_vec)

# ベクトル化版の要約（95% HDI, print利用）
print("\n[Vectorized model] ArviZ summary (95% HDI):")
print(az.summary(idata_vec, var_names=["mu", "sigma"], hdi_prob=0.95))

# トレースと事後分布（英語ラベル）
az.plot_trace(idata_vec, var_names=["mu", "sigma"])
plt.suptitle("Trace Plot (Vectorized Model)", y=1.02)
plt.tight_layout()
plt.show()

az.plot_posterior(idata_vec, var_names=["mu", "sigma"], hdi_prob=0.95)
plt.suptitle("Posterior (Vectorized Model, 95% HDI)", y=1.02)
plt.tight_layout()
plt.show()
